In [205]:
import pandas as pd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [228]:
df_clean = pd.read_csv('data/df_clean.csv')

# selection des features et de la cible
X = df_clean.drop('charges', axis=1)

y = df_clean.charges

display(X)

,age,sex,bmi,children,smoker,region
0,19,female,27.900,0,yes,southwest
1,18,male,33.770,1,no,southeast
2,28,male,33.000,3,no,southeast
3,33,male,22.705,0,no,northwest
4,32,male,28.880,0,no,northwest
...,...,...,...,...,...,...
1332,50,male,30.970,3,no,northwest
1333,18,female,31.920,0,no,northeast
1334,18,female,36.850,0,no,southeast
1335,21,female,25.800,0,no,southwest


In [220]:
# import du csv
df = pd.read_csv('data/df_clean.csv')

# Création du train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, train_size=0.85, random_state=0)

age           int64
sex          object
bmi         float64
children      int64
smoker       object
region       object
dtype: object

In [260]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures, OrdinalEncoder
from sklearn.linear_model import LinearRegression, Lasso,ElasticNet,Ridge
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.preprocessing import FunctionTransformer

# Séparation des colonnes numériques et catégorielles

cat_col = ['region']


num_col = df_clean.select_dtypes(include ='int').columns

ordinal_col= ['sex', 'smoker']

#Transformation de BMI avec FunctionTransformer


# pipeline pour les valeurs numériques
# cela permet de créer une succession d'étapes (ici il n'y en a qu'une)

my_num_pipe = make_pipeline(StandardScaler())

my_cat_pipe = make_pipeline(OneHotEncoder())

my_ordi_pipe = make_pipeline(OrdinalEncoder())

                            
# ColumnTransformer permet de faire des opérations sur des sélections de colonnes

preprocessor = make_column_transformer(
                                       (my_ordi_pipe,ordinal_col),  
                                       (my_cat_pipe,cat_col)
                                       
                                       )


final_preprocessor = make_pipeline(preprocessor,PolynomialFeatures(degree=2),StandardScaler())


# affichage

tab = final_preprocessor.fit(X_train)

cols = tab.get_feature_names_out()

df_transform = pd.DataFrame(final_preprocessor.fit_transform(X_train), columns=cols)
df_transform.head()

ModuleNotFoundError: No module named 'fonction'

In [256]:
final_preprocessor

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('ordinalencoder',
                                                                   OrdinalEncoder())]),
                                                  ['sex', 'smoker']),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('onehotencoder',
                                                                   OneHotEncoder())]),
                                                  ['region'])])),
                ('polynomialfeatures', PolynomialFeatures()),
                ('standardscaler', StandardScaler())])

In [255]:
from sklearn.model_selection import GridSearchCV

model= make_pipeline(final_preprocessor, ElasticNet(max_iter=1000000))

# Définir les valeurs à tester pour alpha et l1_ratio

param_grid = {'elasticnet__alpha': np.arange(50,100,1),    #np.arange(0.01, 0.1,0.001)
              'elasticnet__l1_ratio':[1]}      #np.arange(0.1,1,0.01)


# creation de l'objet GridSearch

grid_search = GridSearchCV(model,param_grid, cv=5)

# recherche des hyperparamètres et Afficher les meilleures valeurs trouvées

grid_search.fit(X_train, y_train)

print("Meilleurs paramètres : ",grid_search.best_params_)

# Utiliser le modèle avec les meilleures valeurs
best_model = grid_search.best_estimator_

best_model.score(X_test, y_test)

print("Score : ", best_model.score(X_test, y_test))

# calcul RSME

rmse= np.sqrt(mean_squared_error(y_test,best_model.predict(X_test)))
print("RSME: ",rmse)


Meilleurs paramètres :  {'elasticnet__alpha': 99, 'elasticnet__l1_ratio': 1}
Score :  0.5665531533691595
RSME:  8319.565984527952


## Regression Linéaire 

In [ ]:

y=df_clean[['charges']]
X = df_clean[['bmi','age','smoker','sex','children']]

# regression lineaire, calcule R2, scatterplot

model = make_pipeline(preprocessor,LinearRegression())
model.fit(X_train,y_train)
model.score(X_train,y_train)
print("R2 (coefficient de détermination): ",model.score(X_test,y_test))


R2 (coefficient de détermination):  0.7321966386775064


## Regression Lasso

In [253]:
model= make_pipeline(final_preprocessor,Lasso(max_iter=1000000))

param_grid={'lasso__alpha':np.arange(0.1,100,10)}

grid_search = GridSearchCV(model,param_grid, cv=5)

grid_search.fit(X_train, y_train)

print("Meilleurs paramètres : ",grid_search.best_params_)

# Utiliser le modèle avec les meilleures valeurs
best_model = grid_search.best_estimator_

best_model.score(X_test, y_test)


Meilleurs paramètres :  {'lasso__alpha': 90.1}


0.5664341681092777

## Regression Ridge

In [248]:
model= make_pipeline(preprocessor,Ridge(max_iter=1000000))
model.fit(X_train,y_train)
model.score(X_test,y_test)

0.5631673819161284